<a href="https://colab.research.google.com/github/plaban1981/Hugging_Face_transformers_topics/blob/main/mosaicml_mpt_7b_storywriter_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -qU -e git+https://github.com/samhavens/just-triton-flash.git#egg=flash_attn 

  Preparing metadata (setup.py) ... done


In [4]:
!pip install -qU  einops transformers

In [6]:
!pip install -qU openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18.0 MB/s eta 0:00:00


## Define the Pipeline

In [9]:
from typing import Any, Dict, Tuple
import warnings

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
#
import datetime
import os
from threading import Event, Thread
from uuid import uuid4

import requests
import torch
from transformers import StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer
#
import torch


In [ ]:
from getpass import getpass
import os
#
os.environ["HF_TOKEN"] = getpass()
# Configuration
HF_TOKEN = os.getenv("HF_TOKEN", None)

··········


In [4]:
from google.colab import drive 
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
model =  AutoModelForCausalLM.from_pretrained(
            "mosaicml/mpt-7b-storywriter",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True)


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-storywriter:
- configuration_mpt.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-storywriter:
- meta_init_context.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-storywriter:
- norm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-storywriter:
- param_init_fns.py
- norm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-storywriter:
- flash_attn_triton.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-storywriter:
- attention.py
- flash_attn_triton.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-storywriter:
- adapt_tokenizer.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-storywriter:
- blocks.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-storywriter:
- hf_prefixlm_converter.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-storywriter:
- modeling_mpt.py
- meta_init_context.py
- param_init_fns.py
- attention.py
- adapt_tokenizer.py
- blocks.py
- hf_prefixlm_converter.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


/root/.cache/huggingface/modules/transformers_modules/mosaicml/mpt-7b-storywriter/6825a30eda8e732334fe2884a0f433f5abff9545/attention.py:157: UserWarning: Using `attn_impl: torch`. If your model does not use `alibi` or `prefix_lm` we recommend using `attn_impl: flash` otherwise we recommend using `attn_impl: triton`.
  warnings.warn('Using `attn_impl: torch`. If your model does not use `alibi` or ' + '`prefix_lm` we recommend using `attn_impl: flash` otherwise ' + 'we recommend using `attn_impl: triton`.')


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/write_story_model/MODEL",
                                             torch_dtype=torch.bfloat16,
                                             trust_remote_code=True)

A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-storywriter:
- configuration_mpt.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-storywriter:
- norm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-storywriter:
- flash_attn_triton.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-storywriter:
- attention.py
- flash_attn_triton.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-storywriter:
- blocks.py
- norm.py
- attention.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-storywriter:
- meta_init_context.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-storywriter:
- hf_prefixlm_converter.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-storywriter:
- adapt_tokenizer.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-storywriter:
- param_init_fns.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-storywriter:
- modeling_mpt.py
- blocks.py
- meta_init_context.py
- hf_prefixlm_converter.py
- adapt_tokenizer.py
- param_init_fns.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


You are using config.init_device='cpu', but you can also use config.init_device="meta" with Composer + FSDP for fast initialization.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/write_story_model/TOKENS')

In [ ]:
tokenizer.save_pretrained("/content/drive/MyDrive/Story_writer")

('/content/drive/MyDrive/Story_writer/tokenizer_config.json',
 '/content/drive/MyDrive/Story_writer/special_tokens_map.json',
 '/content/drive/MyDrive/Story_writer/tokenizer.json')

In [ ]:
import shutil 
shutil.move('/content/drive/MyDrive/write_story_model/tokenizer.json','/content/drive/MyDrive/write_story_model/TOKENS')

'/content/drive/MyDrive/write_story_model/TOKENS/tokenizer.json'

In [10]:
class TextGenerationPipeline:
    def __init__(
        self,
        model_name,
        torch_dtype=torch.bfloat16,
        trust_remote_code=True,
        use_auth_token=None,
    ) -> None:
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch_dtype,
            trust_remote_code=trust_remote_code,
            #use_auth_token=use_auth_token,
            max_seq_len=10240, 
        )

        tokenizer = AutoTokenizer.from_pretrained(
            model_name,
            trust_remote_code=trust_remote_code,
            #use_auth_token=use_auth_token,
        )
        if tokenizer.pad_token_id is None:
            warnings.warn(
                "pad_token_id is not set for the tokenizer. Using eos_token_id as pad_token_id."
            )
            tokenizer.pad_token = tokenizer.eos_token
        tokenizer.padding_side = "left"
        self.tokenizer = tokenizer

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.eval()
        self.model.to(device=device, dtype=torch_dtype)

        self.generate_kwargs = {
            "temperature": 0.8,
            "top_p": 0.95,
            "top_k": 50,
            "max_new_tokens": 512,
            "use_cache": True,
            "do_sample": True,
            "eos_token_id": self.tokenizer.eos_token_id,
            "pad_token_id": self.tokenizer.pad_token_id,
            "repetition_penalty": 1.02,  # 1.0 means no penalty, > 1.0 means penalty, 1.2 from CTRL paper
        }

    def __call__(
        self, instruction: str, **generate_kwargs: Dict[str, Any]
    ) -> Tuple[str, str, float]:
        input_ids = self.tokenizer(instruction, return_tensors="pt").input_ids
        input_ids = input_ids.to(self.model.device)
        gkw = {**self.generate_kwargs, **generate_kwargs}
        with torch.no_grad():
            output_ids = self.model.generate(input_ids, **gkw)
        # # Slice the output_ids tensor to get only new tokens
        # new_tokens = output_ids[0, len(input_ids[0]) :]
        output_tokens = output_ids[0]
        output_text = self.tokenizer.decode(output_tokens, skip_special_tokens=True)
        return output_text


In [39]:
# Define a custom stopping criteria
class StopOnTokens(StoppingCriteria):
    def __init__(self, stop_token_ids):
        self.stop_token_ids = stop_token_ids

    def __call__(self, input_ids: torch.LongTensor, stop_token_ids: list[int], **kwargs) -> bool:
        for stop_id in self.stop_token_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False

In [ ]:
# Define a custom stopping criteria
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_id in [0]:
            if input_ids[0][-1] == stop_id:
                return True
        return False

In [11]:
# Initialize the model and tokenizer
generate = TextGenerationPipeline(
    "mosaicml/mpt-7b-storywriter",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    #use_auth_token=HF_TOKEN,
)
#stop_token_ids = generate.tokenizer.convert_tokens_to_ids(["<|endoftext|>"])

You are using config.init_device='cpu', but you can also use config.init_device="meta" with Composer + FSDP for fast initialization.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

<ipython-input-10-be92ce92e1a5>:23: UserWarning: pad_token_id is not set for the tokenizer. Using eos_token_id as pad_token_id.
  warnings.warn(


In [20]:
import gc
import torch
gc.collect()
# Empty the CUDA cache
torch.cuda.empty_cache()

In [17]:
stop_token_ids = generate.tokenizer.convert_tokens_to_ids(["<|endoftext|>"])

In [15]:

generate.__call__(instruction=instruction)

'# My Finest Hour\n## Chapter 1\n*\nThe sun rose that morning just like any other morning. Bright, obtrusive, and dangerous. Night seemed like the only protection I had anymore, but I knew even that was a just a lie I told myself so I could rest a little easier.\nI wasn\'t the only one to stir as the morning sun crested the horizon. I could hear the clanks and scrapes of others in their tents, who, like me, found it simplest to just sleep in their armor. It ached, but everything ached out here. It was war, after all. I scanned my tent. For all its comforts, I sought but one: my trusty sword. It was a fine example of a forgotten style. \nIt had been forged in a time when swords were meant to be used, not to be hidden away. It was a single-handed weapon, with two edges, though the point was dulled by use.\n\nI pulled it free from under my blankets and threw my cloak over my shoulders as I stepped outside the tent. The others were already gathering, standing at attention for their command

In [19]:
instruction = """I have long wanted to write this tale but only now do I find the courage. It is a tale about my dog named Fido. It is the story of how I learned that Fido could talk!

Our story begins when Fido was still a puppy, only 14 months old. It was a calm summer day and we had just enjoyed a walk."""
#

In [21]:
#
%%time
response = generate.__call__(instruction=instruction)
print(response)

I have long wanted to write this tale but only now do I find the courage. It is a tale about my dog named Fido. It is the story of how I learned that Fido could talk!

Our story begins when Fido was still a puppy, only 14 months old. It was a calm summer day and we had just enjoyed a walk. I took Fido to the park near our house to play with some other dogs.

It was time for us to go home so I picked up Fido in my arms, and off we went. I walked on the path beside the grassy hill, with the tall trees on either side of me. The air was warm and pleasant and I felt happy. I had a basket of cookies in one hand, and my arm was around Fido's neck as he rested his head on my shoulder. Suddenly, I heard something. I stopped and listened.

It sounded like a voice! I could hear it again. Fido's ears pricked up and he gave a little jump, trying to see what I was looking at. I bent down and looked. I saw a small bird perched on the ground. He seemed to be speaking to me!

"Hello, Fido," said the bi

In [19]:
stop_token_ids  = tokenizer.convert_tokens_to_ids(["<|endoftext|>"])

In [22]:
stop = StopOnTokens()

In [23]:
stop

In [44]:
def generate_results(instruction,model,tokenizer):
  #print(f"Prompt \n:{instruction}")
  tokenized_input = tokenizer(instruction, return_tensors="pt")
  #
  stop_token_ids = tokenizer.convert_tokens_to_ids(["<|endoftext|>"])
  #
  stop = StopOnTokens(stop_token_ids=stop_token_ids)
  #
  model = model.to('cuda')
  outputs = model.generate(input_ids=tokenized_input['input_ids'].to('cuda:0'), 
                            max_new_tokens=300, 
                            temperature =0.1,
                            top_k=5, 
                            top_p=0.95,
                            stopping_criteria =StoppingCriteriaList([stop]))
  #
  answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)
  #print(answer[0].rstrip())
  return answer[0].rstrip()

In [13]:
instruction = """# My Finest Hour
## Chapter 1
*
The sun rose that morning just like any other morning. Bright, obtrusive, and dangerous. Night seemed like the only protection I had anymore, but I knew even that was a just a lie I told myself so I could rest a little easier.
I wasn't the only one to stir as the morning sun crested the horizon. I could hear the clanks and scrapes of others in their tents, who, like me, found it simplest to just sleep in their armor. It ached, but everything ached out here. It was war, after all. I scanned my tent. For all its comforts, I sought but one: my trusty sword. It was a fine example of a forgotten style. """


In [49]:
%%time
content = generate_results(instruction,model,tokenizer)
print(content)

# My Finest Hour
## Chapter 1
*
The sun rose that morning just like any other morning. Bright, obtrusive, and dangerous. Night seemed like the only protection I had anymore, but I knew even that was a just a lie I told myself so I could rest a little easier.
I wasn't the only one to stir as the morning sun crested the horizon. I could hear the clanks and scrapes of others in their tents, who, like me, found it simplest to just sleep in their armor. It ached, but everything ached out here. It was war, after all. I scanned my tent. For all its comforts, I sought but one: my trusty sword. It was a fine example of a forgotten style. 
The hilt was a simple, unadorned piece of wood, but the blade was a thing of beauty. It was a long, thin, curved blade, with a slight curve to it. It was a style that had been all but forgotten, but I had found it to be the most effective. It was a style that was perfect for a man like me.

I had been a soldier for a long time, and I had seen many things. I ha

In [45]:
instruction = """I have long wanted to write this tale but only now do I find the courage. It is a tale about my dog named Fido. It is the story of how I learned that Fido could talk!

Our story begins when Fido was still a puppy, only 14 months old. It was a calm summer day and we had just enjoyed a walk."""
#

In [46]:
%%time
content = generate_results(instruction,model,tokenizer)
print(content)

I have long wanted to write this tale but only now do I find the courage. It is a tale about my dog named Fido. It is the story of how I learned that Fido could talk!

Our story begins when Fido was still a puppy, only 14 months old. It was a calm summer day and we had just enjoyed a walk. I was sitting on the porch, reading a book, when Fido came running up to me. He was panting and his tongue was hanging out.

"What is it, Fido?" I asked.

"I have a secret to tell you," he said.

"What is it?" I asked again.

"I have a secret to tell you," he said again.

"What is it?" I asked again.

"I have a secret to tell you," he said again.

"What is it?" I asked again.

"I have a secret to tell you," he said again.

"What is it?" I asked again.

"I have a secret to tell you," he said again.

"What is it?" I asked again.

"I have a secret to tell you," he said again.

"What is it?" I asked again.

"I have a secret to tell you," he said again.

"What is it?" I asked again.

"I have a secret to t

In [ ]:
def generate_results(instruction):
  print(f"Prompt \n:{instruction}")
  tokenized_input = generate.tokenizer(instruction, return_tensors="pt")
  #
  stop = StopOnTokens()
  #
  outputs = generate.model.generate(input_ids=tokenized_input['input_ids'].to('cuda:0'), 
                            max_new_tokens=200, 
                            temperature =0.1,
                            top_k=5, 
                            top_p=0.95,
                            stopping_criteria =StoppingCriteriaList([stop]))
  #
  answer = generate.tokenizer.batch_decode(outputs, skip_special_tokens=True)
  print(answer[0].rstrip())
  return answer[0].rstrip()

In [ ]:
instruction = """# My Finest Hour
## Chapter 1
*
The sun rose that morning just like any other morning. Bright, obtrusive, and dangerous. Night seemed like the only protection I had anymore, but I knew even that was a just a lie I told myself so I could rest a little easier.
I wasn't the only one to stir as the morning sun crested the horizon. I could hear the clanks and scrapes of others in their tents, who, like me, found it simplest to just sleep in their armor. It ached, but everything ached out here. It was war, after all. I scanned my tent. For all its comforts, I sought but one: my trusty sword. It was a fine example of a forgotten style. """


In [ ]:
content = generate_results(instruction)
print(content)

Prompt 
:# My Finest Hour
## Chapter 1
*
The sun rose that morning just like any other morning. Bright, obtrusive, and dangerous. Night seemed like the only protection I had anymore, but I knew even that was a just a lie I told myself so I could rest a little easier.
I wasn't the only one to stir as the morning sun crested the horizon. I could hear the clanks and scrapes of others in their tents, who, like me, found it simplest to just sleep in their armor. It ached, but everything ached out here. It was war, after all. I scanned my tent. For all its comforts, I sought but one: my trusty sword. It was a fine example of a forgotten style. 
# My Finest Hour
## Chapter 1
*
The sun rose that morning just like any other morning. Bright, obtrusive, and dangerous. Night seemed like the only protection I had anymore, but I knew even that was a just a lie I told myself so I could rest a little easier.
I wasn't the only one to stir as the morning sun crested the horizon. I could hear the clanks 

In [ ]:
import transformers
transformers.__version__

'4.29.2'

In [ ]:
import einops 
einops.__version__

'0.6.1'

In [ ]:
import torch
torch.__version__

'2.0.1+cu118'

In [ ]:
import torchvision
torchvision.__version__

'0.15.2+cu118'